https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ko

In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.4.1


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation="relu", input_shape=(784, )),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(optimizer="adam",
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=["accuracy"])
    
    return model

model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

Epoch 1/10
32/32 [==============================] - 1s 22ms/step - loss: 1.5263 - accuracy: 0.5415 - val_loss: 0.6593 - val_accuracy: 0.8060

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 8ms/step - loss: 0.4096 - accuracy: 0.8878 - val_loss: 0.5312 - val_accuracy: 0.8330

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 8ms/step - loss: 0.2570 - accuracy: 0.9341 - val_loss: 0.4837 - val_accuracy: 0.8460

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 8ms/step - loss: 0.1942 - accuracy: 0.9546 - val_loss: 0.4635 - val_accuracy: 0.8520

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 9ms/step - loss: 0.1513 - accuracy: 0.9668 - val_loss: 0.4413 - val_accuracy: 0.8600

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
32/32 [==============================] - 0s

In [5]:
ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [6]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Accuracy of untrained model : {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.2707 - accuracy: 0.1590
Accuracy of untrained model : 15.90%


In [7]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Accuracy of reconstructed model : {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4190 - accuracy: 0.8690
Accuracy of reconstructed model : 86.90%


In [8]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [9]:
ls {checkpoint_dir}

checkpoint                        cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index                cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index                cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index                cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index                cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index                cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [10]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [11]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Accuracy of reconstructed model : {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4784 - accuracy: 0.8730
Accuracy of reconstructed model : 87.30%


In [12]:
model.save_weights("./checkpoints/my_checkpoint")

model = create_model()

model.load_weights("./checkpoints/my_checkpoint")

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Accuracy of reconstructed model : {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4784 - accuracy: 0.8730
Accuracy of reconstructed model : 87.30%


In [13]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model
model.save("saved_model/my_model")

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.6246 - accuracy: 0.4415
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4529 - accuracy: 0.8571
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.3483 - accuracy: 0.9113
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2044 - accuracy: 0.9562
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1582 - accuracy: 0.9619
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [14]:
!ls saved_model

!ls saved_model/my_model

my_model
assets	saved_model.pb	variables


In [15]:
new_model = tf.keras.models.load_model("saved_model/my_model")

new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Accuracy of reconstructed model : {:5.2f}%".format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4243 - accuracy: 0.8650
Accuracy of reconstructed model : 86.50%
(1000, 10)


In [17]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save("my_model.h5")

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.5965 - accuracy: 0.5269
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4262 - accuracy: 0.8841
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.3253 - accuracy: 0.9141
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2002 - accuracy: 0.9576
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1658 - accuracy: 0.9684


In [18]:
new_model = tf.keras.models.load_model("my_model.h5")

new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [19]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Accuracy of reconstructed model : {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4187 - accuracy: 0.8630
Accuracy of reconstructed model : 86.30%
